# Cross validation of the ALS method using Spark
We refer to this tutorial: http://spark.apache.org/docs/latest/ml-tuning.html#example-model-selection-via-cross-validation

The tutorial was not clear: we used SKLearn to split the dataset

# Import

In [17]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import itertools
from pyspark.mllib.recommendation import ALS
import math

%matplotlib inline
%load_ext autoreload
%autoreload 2

pd.options.display.max_columns = 100


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from pyspark.sql.functions import col

In [19]:
from helpers_cross_validation import KFoldIndexes,CrossValidation

In [20]:
from helpers_als import get_predictions_ALS

In [21]:
import helpers

In [22]:
import random


# Models

In [7]:
class ALSModel:
    
    def __init__(self):
        pass
    
    def fit(self,data,**arg):
        self.model = ALS.train(data, **arg)
    
    def predict(self,data):
        data_for_preditions=data.map(lambda x: (x[0], x[1]))
        self.predictions = self.model.predictAll(data_for_preditions).map(lambda r: ((r[0], r[1]), r[2]))
    
    def evaluate(self,data):
        rates_and_preds = data.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(self.predictions)
        error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
        return error

# Dataframe creation

In [8]:
train = helpers.load_csv()
train.head()

,UserID,MovieID,Rating
0,44,1,4
1,61,1,3
2,67,1,4
3,72,1,3
4,86,1,5


In [9]:
# First, we transform it using sqlContect
train_rdd = helpers.to_rdd(sqlContext, train)
train_rdd.take(3)

[Row(UserID=44, MovieID=1, Rating=4),
 Row(UserID=61, MovieID=1, Rating=3),
 Row(UserID=67, MovieID=1, Rating=4)]

In [10]:
train.shape[0]

1176952

# Cross validation

In [ ]:
cv=CrossValidation(train,4,True,sc)

In [ ]:
ranks = [8]
lambdas = [0.08,0.081,0.082,0.083,0.084,0.085,0.086,0.087,0.088,0.089,0.09]
numIters = [24]
nbr_models = len(ranks)*len(lambdas)*len(numIters)

bestModel = None
bestValidationRmse = float("inf")
bestRank = 0
bestLambda = -1.0
bestNumIter = -1

In [ ]:
bestLambda

In [ ]:
i = 0
for rank, lmbda, numIter in itertools.product(ranks, lambdas, numIters):
    try:
        print(rank,lmbda,numIter)
        validationRmse = cv.evaluate(ALSModel(),rank=rank,lambda_=lmbda, iterations=numIter, nonnegative=True)
        validationRmse = np.mean(validationRmse)
        print("Model %i/%i: RMSE (validation) = %f" %(i+1, nbr_models, validationRmse))
        print("  Trained with rank = %d, lambda = %.1f, and numIter = %d." % (rank, lmbda, numIter))
        print("")
        if (validationRmse < bestValidationRmse):
#             bestModel = model
            bestValidationRmse = validationRmse
            bestRank = rank
            bestLambda = lmbda
            bestNumIter = numIter
    except:
        print("Model %i/%i failed!" %(i+1, nbr_models))
        print("  Parameters: rank = %d, lambda = %.1f, and numIter = %d." % (rank, lmbda, numIter))

    i += 1
    
# Evaluate the best model on the training set
print("The best model was trained with rank = %d and lambda = %.1f, " % (bestRank, bestLambda) \
  + "and numIter = %d, and its RMSE on the training set is %f" % (bestNumIter, bestValidationRmse))

# # Evaluate the best model on the test set
# testRmse = computeRMSE(bestModel, test_for_predict_RDD, test_RDD)
# print("RMSE on the test set: %f"%(testRmse))


# Prepare data for submission

In [ ]:
test = pd.read_csv('../data/sampleSubmission.csv')
test.head()

In [ ]:
# Prepare test for RDD
test_prep = test
test_prep['UserID'] = test_prep['Id'].apply(lambda x: int(x.split('_')[0][1:]))
test_prep['MovieID'] = test_prep['Id'].apply(lambda x: int(x.split('_')[1][1:]))
test_prep['Rating'] = test_prep['Prediction']
test_prep = test_prep.drop(['Prediction', 'Id'], axis=1)
test_prep.head()

In [ ]:
# First, we transform it using sqlContect
test_sql = sqlContext.createDataFrame(test_prep)
test_rdd = test_sql.rdd
test_rdd.take(3)

In [ ]:
bestModel=ALSModel()
bestModel.fit(train_rdd,rank=8,lambda_=0.081, iterations=24, nonnegative=True)

In [ ]:
bestModel.predict(test_rdd)
predictions=bestModel.predictions
# predictions = bestModel.predictAll(test_RDD_Kaggle).map(lambda r: ((r[0], r[1]), r[2]))

In [ ]:
predictions.take(3)

In [ ]:
pred_df = predictions.toDF().toPandas()

In [ ]:
pred_df['UserID'] = pred_df['_1'].apply(lambda x: x['_1'])
pred_df['MovieID'] = pred_df['_1'].apply(lambda x: x['_2'])
pred_df['Rating'] = pred_df['_2']
pred_df = pred_df.drop(['_1', '_2'], axis=1)
pred_df.head()

In [ ]:
pred_df = pred_df.sort_values(by=['MovieID', 'UserID'])
pred_df.head()

In [ ]:
pred_df.index = range(len(pred_df))

In [ ]:
test['Prediction'] = pred_df['Rating']


In [ ]:
test.head()

In [ ]:
test = test.drop(['UserID', 'MovieID', 'Rating'], axis=1)


In [ ]:
test.to_csv('pred_pyspark_als.csv', index=False)

# Test with get_predictions_ALS function

In [11]:
import random

In [12]:
train_index=random.sample(range(1176952),1000000)
train_index.sort()

In [13]:
test_index=list(set(range(1176952))-set(train_index))
test_index.sort()

In [14]:
training=train.loc[train_index]

In [15]:
testing=train.loc[test_index]

In [23]:
get_predictions_ALS(training,testing,sc,rank=8,lambda_=0.081, iterations=24, nonnegative=True)

,UserID,MovieID,Rating
0,182,1,3.260075
1,670,1,3.177308
2,872,1,2.977831
3,967,1,3.484442
4,981,1,3.349462
5,1034,1,3.298134
6,1107,1,3.195127
7,1230,1,3.288920
8,1489,1,3.357065
9,2090,1,3.041825
